## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Severo-Kurilsk,RU,50.6789,156.1250,56.34,88,99,9.84,overcast clouds,2022-07-19 00:04:18
1,1,Busselton,AU,-33.6500,115.3333,51.26,84,0,7.67,clear sky,2022-07-19 00:04:18
2,2,Hobart,AU,-42.8794,147.3294,46.26,83,75,11.50,broken clouds,2022-07-19 00:00:12
3,3,Killin,US,33.6123,-91.7412,93.29,52,0,5.75,clear sky,2022-07-19 00:04:19
4,4,Padang,ID,-0.9492,100.3543,69.85,73,39,4.05,scattered clouds,2022-07-19 00:04:19


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
11,11,Bubaque,GW,11.2833,-15.8333,79.30,85,99,10.67,light rain,2022-07-19 00:04:22
13,13,Hirado,JP,33.3597,129.5531,77.40,91,96,21.52,moderate rain,2022-07-19 00:04:22
16,16,La Grande,US,45.3246,-118.0877,80.01,31,0,7.52,clear sky,2022-07-19 00:04:23
22,22,Buala,SB,-8.1450,159.5921,85.12,66,36,6.91,light rain,2022-07-19 00:04:26
31,31,Mae Hong Son,TH,19.3003,97.9685,76.42,95,100,1.61,light rain,2022-07-19 00:04:29
34,34,Kailua,US,21.4022,-157.7394,86.02,75,75,16.11,broken clouds,2022-07-19 00:04:29
36,36,Riachao,BR,-7.3619,-46.6172,77.34,39,25,4.07,scattered clouds,2022-07-19 00:04:30
37,37,Avarua,CK,-21.2078,-159.7750,75.25,73,75,9.22,broken clouds,2022-07-19 00:04:30
41,41,Sioux Lookout,CA,50.1001,-91.9170,84.40,38,30,5.08,scattered clouds,2022-07-19 00:04:32
42,42,Albany,US,42.6001,-73.9662,76.73,88,100,8.61,overcast clouds,2022-07-19 00:04:28


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
Date                   0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
11,Bubaque,GW,79.30,light rain,11.2833,-15.8333,
13,Hirado,JP,77.40,moderate rain,33.3597,129.5531,
16,La Grande,US,80.01,clear sky,45.3246,-118.0877,
22,Buala,SB,85.12,light rain,-8.1450,159.5921,
31,Mae Hong Son,TH,76.42,light rain,19.3003,97.9685,
34,Kailua,US,86.02,broken clouds,21.4022,-157.7394,
36,Riachao,BR,77.34,scattered clouds,-7.3619,-46.6172,
37,Avarua,CK,75.25,broken clouds,-21.2078,-159.7750,
41,Sioux Lookout,CA,84.40,scattered clouds,50.1001,-91.9170,
42,Albany,US,76.73,overcast clouds,42.6001,-73.9662,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print(hotels["results"][0]["name"])
    except (IndexError):
        print("Hotel not found... skipping.")
        hotel_df.loc[index, "Hotel Name"] = np.NaN

Casa Dora, Bubaque
Samson Hotel
Travelodge by Wyndham La Grande
Maringe Lagoon Lodge
Baiyoke Chalet Hotel
Sheffield House Bed & Breakfast
Pousada Nordestina
Paradise Inn
Knobby's Fly-In Camps
Hotel not found... skipping.
Hôtel Le Dronmi
Hotel not found... skipping.
Leumerus Bungalows
Hôtel La Giralda
Isles Sunset Lodge
Embassy Suites by Hilton Portland Downtown
Mini Central
Hotel not found... skipping.
Guest House Ruslan Nurata & Tours
Villa Enata
Puri Brata
Scoop Guest House
Umit
Hotel Yugaf Inn Okinawa
Hotel Citra
Waigo Splash Hotel
Carra Valla Inn
Cititel Penang
Holiday Inn St Louis SW - Route 66, an IHG Hotel
Shimoda Tokyu Hotel
Burntwood Hotel
Hotel not found... skipping.
Premier Inn Liverpool City Centre (Moorfields) hotel
The Freeport Inn and Marina
Ikathouse
Grage Hotel Bengkulu
Katsuura Hotel Mikazuki
RAMA CHANDRA REST HOUSE
Naifaru Residence Guest House
Courtyard by Marriott Danbury
Complexe touristique haje bezine
Mövenpick Qassim
Hotel not found... skipping.
Hotel Uribia
Sh

In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)
#clean_hotel_df.tail(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
11,Bubaque,GW,79.30,light rain,11.2833,-15.8333,"Casa Dora, Bubaque"
13,Hirado,JP,77.40,moderate rain,33.3597,129.5531,Samson Hotel
16,La Grande,US,80.01,clear sky,45.3246,-118.0877,Travelodge by Wyndham La Grande
22,Buala,SB,85.12,light rain,-8.1450,159.5921,Maringe Lagoon Lodge
31,Mae Hong Son,TH,76.42,light rain,19.3003,97.9685,Baiyoke Chalet Hotel
34,Kailua,US,86.02,broken clouds,21.4022,-157.7394,Sheffield House Bed & Breakfast
36,Riachao,BR,77.34,scattered clouds,-7.3619,-46.6172,Pousada Nordestina
37,Avarua,CK,75.25,broken clouds,-21.2078,-159.7750,Paradise Inn
41,Sioux Lookout,CA,84.40,scattered clouds,50.1001,-91.9170,Knobby's Fly-In Camps
43,Cayenne,GF,79.09,broken clouds,4.9333,-52.3333,Hôtel Le Dronmi


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))